In [1]:
from dataclasses import dataclass, field

def manhattan(a, b):
    return sum(abs(ai - bi) for ai, bi in zip(a, b))

@dataclass
class Sensor:
    coords: tuple[int, int]
    beacon: tuple[int, int]
    distance: int = field(init=False)
    
    def __post_init__(self):
        self.distance = manhattan(self.coords, self.beacon)
        
    def is_covered(self, p: tuple[int, int]) -> bool:
        return manhattan(self.coords, p) <= self.distance

In [2]:
import re

regex_sensor = re.compile(r"Sensor at x=(?P<sx>-?\d+), y=(?P<sy>-?\d+): closest beacon is at x=(?P<bx>-?\d+), y=(?P<by>-?\d+)")

sensors = []
with open("Day15.txt") as file:
    for line in file:
        if match := regex_sensor.match(line):
            sx, sy, bx, by = map(int, match.groups())
            sensors.append(Sensor((sx, sy), (bx, by)))

In [3]:
%%time
empty, covered, ty = set(), set(), 2_000_000
for sensor in sensors:
    covered.add(sensor.coords)
    covered.add(sensor.beacon)
    (sx, sy), d = sensor.coords, sensor.distance
    if (delta := d - abs(sy - ty)) >= 0:
        for x in range(sx - delta, sx + delta + 1):
            empty.add((x, ty))
len(empty - covered)

CPU times: user 2.45 s, sys: 544 ms, total: 2.99 s
Wall time: 3.01 s


5144286

In [4]:
%%time
def coverage(y):
    for sensor in sensors:
        (sx, sy), d = sensor.coords, sensor.distance
        if (delta_y := abs(sy - y)) > d:
            continue
        delta_x1, delta_x2 = sx - (d - delta_y), sx + (d - delta_y)
        yield delta_x1, delta_x2

def solver(limit=4_000_000):
    for y in range(limit):
        max_x = 0
        for x1, x2 in sorted(coverage(y)):
            if x1 > max_x + 1:
                return (x1 - 1) * limit + y
            max_x = max(max_x, x2)
            if max_x > limit:
                break

solver()

CPU times: user 25.6 s, sys: 1.85 ms, total: 25.6 s
Wall time: 25.7 s


10229191267339